In [ ]:
!pip install llamaapi -q

In [17]:
from llamaapi import LlamaAPI
import json
import os
# Replace 'Your_API_Token' with your actual API token
llama = LlamaAPI('LL-69WCQLo6Rh6ucWGD5jMhWcxeMJZsKikJWlBUfzVys7K8KJw9gKLbokgLxn7zudlp')

In [10]:
# API Request JSON Cell
api_request_json = {
  "model": "llama3-70b",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}

# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1721132565,
  "model": "llama3-70b",
  "usage": {
    "prompt_tokens": 78,
    "completion_tokens": 76,
    "total_tokens": 154
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Lllovely llhuman, llhappy llto llhear llfrom llyou! Llwhat llcan llI llhelp llyou llwith lloday? Llperhaps llwe llcan lllearn llabout llthe lllatest ll llama llfashion lltrends llor llfind llthe llbest llspots llfor llgrazing?",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}


In [12]:
reqDesc = """
for every sentence(lit-x) that I will give you, I need to help me to return the SVO as Spacy do for this sentence, like this structure:
{
"lit-1": {
      "SVO_relationships": [
        {
          "subject": "the fake, country-wet freshness",
          "verb": "evaporated",
          "stem": "evaporate",
          "object": "like the tail end of a sweet dream"
        },
        {
          "subject": "the hot streets",
          "verb": "wavered",
          "stem": "waver",
          "object": "in the sun"
        },
        {
          "subject": "I",
          "verb": "kept hearing",
          "stem": "hear",
          "object": "about the Rosenbergs over the radio and at the office"
        },
        {
          "subject": "I",
          "verb": "couldn't get",
          "stem": "get",
          "object": "them out of my mind"
        }
      ],
      "total_SVOs": 4
  },

  "lit-2": {...
}
are you ready?
"""

In [15]:
categories = {
    "literature": "Lit",
    "medical-articles": "Med",
    "moviescripts": "Mov",
    "news-press-releases": "NPR",
}
def iterate_category(category_name, category_prefix, error_log_path):
    folder_path = f'E:/workspaces/PythonWS/Selenium-python/chatGPT/myDataset/{category_name}'
    files = sorted(os.listdir(folder_path))[51:100]
    all_data = {}
    errors = []
    output_json_temp = f'E:/workspaces/PythonWS/Selenium-python/chatGPT/myResult/Using_LLAMA3/{category_name}.json'
    os.makedirs(os.path.dirname(output_json_temp), exist_ok=True)
    with open(output_json_temp, 'w', encoding='utf-8') as json_file:
        for idx, file_name in enumerate(files, start=1):
            file_path = os.path.join(folder_path, file_name)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    req = text
                    time.sleep(10)
                    resp = make_gpt_request(req)

                    api_request_json = {"model": "llama3-70b",
                      "messages": [
                    {"role": "system", "content": reqDesc},
                    {"role": "user", "content": req},
                      ]
                        }
                
                    # Make your request and handle the response
                    resp = llama.run(api_request_json)

                    #print(resp)
                    svos = resp
                    if svos:
                        all_data[f"{category_prefix}-{idx}"] = {
                            "SVO_relationships": svos,
                            "total_SVOs": len(svos)
                        }
                    else:
                        errors.append(f"No SVOs found in file {file_name} ({category_prefix}-{idx})")
                    print(all_data)
                    json.dump(all_data, json_file, indent=4, ensure_ascii=False)
            except Exception as e:
                errors.append(f"Error processing file {file_name} ({category_prefix}-{idx}): {str(e)}")
    output_json_path = f'E:/workspaces/PythonWS/Selenium-python/chatGPT/myResult/Using_LLAMA3/{category_name}.json'
    os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
    with open(output_json_path, 'w', encoding='utf-8') as json_file2:
        json.dump(all_data, json_file2, indent=4, ensure_ascii=False)
    print(f"SVOs for {category_name} saved to {output_json_path}")

    # Write errors to log file
    with open(error_log_path, 'a', encoding='utf-8') as log_file:
        for error in errors:
            log_file.write(error + '\n')
    print(f"Errors for {category_name} logged to {error_log_path}")

In [18]:
# Error log path
error_log_path = 'E:/workspaces/PythonWS/Selenium-python/chatGPT/myDataset/myResult/Using_LLAMA3/error_log.txt'
os.makedirs(os.path.dirname(error_log_path), exist_ok=True)
# Process each category
for category, prefix in categories.items():
    print(category)
    iterate_category(category, prefix, error_log_path)

literature
SVOs for literature saved to E:/workspaces/PythonWS/Selenium-python/chatGPT/myResult/Using_LLAMA3/literature.json
Errors for literature logged to E:/workspaces/PythonWS/Selenium-python/chatGPT/myDataset/myResult/Using_LLAMA3/error_log.txt
medical-articles
SVOs for medical-articles saved to E:/workspaces/PythonWS/Selenium-python/chatGPT/myResult/Using_LLAMA3/medical-articles.json
Errors for medical-articles logged to E:/workspaces/PythonWS/Selenium-python/chatGPT/myDataset/myResult/Using_LLAMA3/error_log.txt
moviescripts
SVOs for moviescripts saved to E:/workspaces/PythonWS/Selenium-python/chatGPT/myResult/Using_LLAMA3/moviescripts.json
Errors for moviescripts logged to E:/workspaces/PythonWS/Selenium-python/chatGPT/myDataset/myResult/Using_LLAMA3/error_log.txt
news-press-releases
SVOs for news-press-releases saved to E:/workspaces/PythonWS/Selenium-python/chatGPT/myResult/Using_LLAMA3/news-press-releases.json
Errors for news-press-releases logged to E:/workspaces/PythonWS/Se

In [ ]:
api_request_json = {
  "messages": [
    {"role": "user", "content": "Extract the desired information from the following passage.:\n\nHi!"},
  ],
  "functions": [
        {'name': 'information_extraction',
         'description': 'Extracts the relevant information from the passage.',
         'parameters': {
             'type': 'object',
             'properties': {
                 'sentiment': {
                    'title': 'sentiment',
                    'type': 'string',
                    'description': 'the sentiment encountered in the passage'
                    },
                 'aggressiveness': {
                    'title': 'aggressiveness',
                    'type': 'integer',
                    'description': 'a 0-10 score of how aggressive the passage is'
                    },
                 'language': {
                    'title': 'language',
                    'type': 'string',
                    'description': 'the language of the passage'
                    }
             },
             'required': ['sentiment', 'aggressiveness', 'language']
         }
      }
    ],
  "stream": False,
  "function_call": {"name": "information_extraction"},
}

# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))